# Segmentation

In [209]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
import PIL.Image as Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np



class Segmentation(object):
    @staticmethod
    def segmentCut(image, document_text):
        segmentFile=open(document_text, 'r')
        
        lines = segmentFile.readlines()
        listOfWordPics=list()
        
        for line in lines:
            left0, top0, right0, bottom0, content = line.split(" ")
            left=int(left0)
            right=int(right0)
            top=int(top0)
            bottom=int(bottom0)
            
            word=image.crop((left, top, right, bottom))
            listOfWordPics.append((word,content))
        
        return listOfWordPics




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# BagOfFeatures

### Import-Section

In [210]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import scipy.signal
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import PIL.Image as Image

# matplotlib.use('Qt5Agg')

import cv2
import numpy as np
import os
import pickle




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1. Für den gesamten Datensatz:
- Graustufen-Darstellung erzeugen
- Auf Vektoren reduzieren (lokale Bild-Deskriptoren - SIFT)

2. Visual Vocabulary finden 
- Clustern
- Gewichten

3. Quantisieren
- Abbildung von Deskriptoren auf Centruiden
- Wie oft kommt ein vis. Word aus dem vis. Voc in dem Bild vor
-> Histogramm

4. Distanzen zwischen den Histogrammen vergleichen

In [211]:
from common import features
from scipy.cluster.vq import kmeans2
documentSegmentation=os.path.join('GT', '2700270.gtp')
documentImage=os.path.join('pages', '2700270.png')
image=Image.open(documentImage)


wordImageList=Segmentation.segmentCut(image, documentSegmentation)
wordImageList

[(<PIL.Image.Image image mode=L size=139x66 at 0x17DB6CC4970>, '270\n'),
 (<PIL.Image.Image image mode=L size=248x103 at 0x17DB6CC6230>, 'letters\n'),
 (<PIL.Image.Image image mode=L size=252x80 at 0x17DB6CC5E10>, 'orders\n'),
 (<PIL.Image.Image image mode=L size=207x80 at 0x17DB6CC5990>, 'and\n'),
 (<PIL.Image.Image image mode=L size=551x98 at 0x17DB6CC4D30>,
  'instructions\n'),
 (<PIL.Image.Image image mode=L size=246x100 at 0x17DB6CC70A0>, 'october\n'),
 (<PIL.Image.Image image mode=L size=146x74 at 0x17DB6CC6050>, '1755\n'),
 (<PIL.Image.Image image mode=L size=159x123 at 0x17DB6CC4C40>, 'only\n'),
 (<PIL.Image.Image image mode=L size=131x104 at 0x17DB6CC63E0>, 'for\n'),
 (<PIL.Image.Image image mode=L size=143x87 at 0x17DB6CC4760>, 'the\n'),
 (<PIL.Image.Image image mode=L size=327x104 at 0x17DB6CC57E0>, 'publick\n'),
 (<PIL.Image.Image image mode=L size=156x67 at 0x17DB6CC5420>, 'use\n'),
 (<PIL.Image.Image image mode=L size=251x64 at 0x17DB6CC5A80>, 'unless\n'),
 (<PIL.Image.Im

In [212]:
#plt.figure()
#wordImageList[5][0].show()

#document_image_filename = os.path.join('pages', wordnames[i])
img = cv2.imread(wordImageList[5][0])
gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
frames, desc = features.compute_sift_descriptors(gray,step_size = 15,cell_size = 3)

TypeError: Can't convert object to 'str' for 'filename'

In [ ]:
from common import features
from scipy.cluster.vq import kmeans2

#init.
wordnames=['2700270_for.png','2700270_only.png','2700270_the.png','2700270_you.png','2700270_the1.png']
step_size = 15
cell_size = 3
n_centroids = 4
histograms=[]

# ?müssen wir wir erst wieder die visu. Words finden (scannen aller Bilder)
#For all Images:
#- Change Color scale to gray
#- build SIFT representation
for i in wordImageList:
    document_image_filename = os.path.join('pages', wordnames[i])
    img = cv2.imread(document_image_filename)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

    #sift = cv2.SIFT_create()
    #kp = sift.detect(gray,None)
    #desc = sift.compute(gray,kp)

    frames, desc = features.compute_sift_descriptors(gray,step_size = 15,cell_size = 3)

    #Open CV nutzen 

    #clustering of SIFT descriptors
    # ?sollten wir nicht villeicht lieber mit den Zentruiden arbeiten?
    # Wurde schon auf 8 Hautrichtungen quantisiert?
    centroid, labels = kmeans2(desc, n_centroids, iter=20, minit='points')

    #Warum erstellen wir wieder ein Histogramm?
    # Reicht es nicht aus mit den Zentruiden zu arbeiten?
    hist = np.bincount(labels)
    histograms.append(hist)



train_labels=np.array([['for'],
                       ['only'],
                       ['the'],
                       ['you']])

In [ ]:
test=np.array([[ 5, 22, 17, 16]])

In [ ]:

from common.classification import KNNClassifier
import scipy.spatial.distance as ds


knn_classifier = KNNClassifier(k_neighbors=3, metric='euclidean')
knn_classifier.estimate(train_samples=histograms[:3],train_labels=train_labels)
knn_test_labels = knn_classifier.classify2(test_samples=test)
knn_test_labels





In [ ]:
"""

def classify1k(train_samples, train_labels,test_samples, metric='euclidean'):
    distances = ds.cdist(train_samples, test_samples, metric=metric)
    test_labels_indices = np.argmin(distances, axis=0)
    return train_labels[test_labels_indices]


result=classify1k(train_samples=histograms[:3],train_labels=train_labels,test_samples=test,metric='euclidean')
result
"""


'''




from matplotlib.patches import Circle, Rectangle
from matplotlib.lines import Line2D

draw_descriptor_cells = True
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(im_arr, cmap=cm.get_cmap('Greys_r'))
ax.autoscale(enable=False)
colormap = cm.get_cmap('jet')
desc_len = cell_size * 4
for (x, y), label in zip(frames, labels):
    color = colormap(label / float(n_centroids))
    circle = Circle((x, y), radius=1, fc=color, ec=color, alpha=1)
    rect = Rectangle((x - desc_len / 2, y - desc_len / 2),
                     desc_len, desc_len, alpha=0.08, lw=1)
    ax.add_patch(circle)
    if draw_descriptor_cells:
        for p_factor in [0.25, 0.5, 0.75]:
            offset_dyn = desc_len * (0.5 - p_factor)
            offset_stat = desc_len * 0.5
            line_h = Line2D((x - offset_stat, x + offset_stat),
                            (y - offset_dyn, y - offset_dyn), alpha=0.08, lw=1)
            line_v = Line2D((x - offset_dyn, x - offset_dyn),
                            (y - offset_stat, y + offset_stat), alpha=0.08, lw=1)
            ax.add_line(line_h)
            ax.add_line(line_v)
    ax.add_patch(rect)
plt.show()


hist = np.bincount(labels)
hist


y_values=hist
x_pos = np.arange(len(y_values))
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(x_pos, y_values, width=0.9, align='center', alpha=0.4,color=[colormap(i / float(n_centroids)) for i in x_pos])

ax.set_xticks(x_pos)
plt.show()
'''





